In [26]:
import ee
# ee.Authenticate()
ee.Initialize()

In [165]:
# load necessary packages
import numpy as np
import pandas as pd
# for visualization 
import altair as alt

# Getting the data from Landsat data collections 

# Preprocessing the landsat data collections: 1. rename the bands and build the bitmasks for the pixels we do not want

In [28]:
# Functions to rename Landsats
#landsat 1-2-3-4M-5M have only 6 bands 
def renameL1_5M(img):
    return img.rename(['GREEN', 'RED', 'NIR', 'NIR2','QA_PIXEL','QA_RADSAT'])

# landsat 4T-5T-7 have similar bands with the following meanings 
def renameL457(img):
    return img.rename(['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1',
        'SWIR2', 'TEMP1', 'ATMOS_OPACITY', 'QA_CLOUD','ATRAN', 'CDIST','DRAD', 
        'EMIS', 'EMSD', 'QA', 'TRAD', 'URAD','QA_PIXEL','QA_RADSAT'])

# landsat 8 has an extra band in 0 place, so the first one is not blue
def renameL89(img):
    return img.rename(['AEROS', 'BLUE', 'GREEN', 'RED', 'NIR',
        'SWIR1','SWIR2', 'TEMP1', 'QA_AEROSOL', 'ATRAN', 'CDIST','DRAD', 'EMIS',
        'EMSD', 'QA', 'TRAD', 'URAD', 'QA_PIXEL', 'QA_RADSAT'])



In [29]:
# call and rename all the Landsat collections
l1 = ee.ImageCollection("LANDSAT/LM01/C02/T1").map(renameL1_5M)
l2 = ee.ImageCollection("LANDSAT/LM02/C02/T1").map(renameL1_5M)
l3 = ee.ImageCollection("LANDSAT/LM03/C02/T1").map(renameL1_5M)
lM4 = ee.ImageCollection("LANDSAT/LM04/C02/T1").map(renameL1_5M)
lM5 = ee.ImageCollection("LANDSAT/LM05/C02/T1").map(renameL1_5M)
lT4 = ee.ImageCollection('LANDSAT/LT04/C02/T1_L2').map(renameL457)
lT5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').map(renameL457)
l7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').map(renameL457)
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').map(renameL89)
l9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").map(renameL89)

In [30]:
# Functions to add masks bands based on QA_PIXEL bit info
def addMask(img):
    """
    img.select('QA_PIXEL').bitwiseAnd(64)
    this piece checks wether the 6th bit of QA_PIXEL is 1 or not. from doc we have
    Bit 6: Clear
        0: Cloud or Dilated Cloud bits are set
        1: Cloud and Dilated Cloud bits are not set
    so want to be 1
    """
    clear = img.select('QA_PIXEL').bitwiseAnd(64).neq(0)
    clear = clear.updateMask(clear).rename(['pxqa_clear'])

    water = img.select('QA_PIXEL').bitwiseAnd(128).neq(0)
    water = water.updateMask(water).rename(['pxqa_water'])

    cloud_shadow = img.select('QA_PIXEL').bitwiseAnd(16).neq(0)
    cloud_shadow = cloud_shadow.updateMask(cloud_shadow).rename(['pxqa_cloudshadow'])

    snow = img.select('QA_PIXEL').bitwiseAnd(32).eq(0)
    snow = snow.updateMask(snow).rename(['pxqa_snow'])

    masks = ee.Image.cat([clear, water, cloud_shadow, snow])

    return img.addBands(masks)


# masking out clouds function 
def maskQAClear(img):
    return img.updateMask(img.select('pxqa_clear'))


# masking out snow function 
def maskQASnow(img):
    return img.updateMask(img.select('pxqa_snow'))

In [31]:
# Merge Landsats
landsat = ee.ImageCollection(l1.merge(l2).merge(l3).merge(lM4).merge(lM5).merge(lT4).merge(lT5).merge(l7).merge(l8).merge(l9))
# .sort('system:time_start')

# now we first add the masks we have built and mask out clouds and snows 
landsat = landsat.map(addMask).map(maskQAClear).map(maskQASnow)

Now we have all the Landsat data collections merged and snowy and cloudy pixels are maked out. We now are ready to add other bands we want to calculate like NDVI. 

# Adding other bands

## NDVI

In [32]:
#Function to add NDVI as a band.
def add_NDVI(img):
    
    NDVI = img.normalizedDifference(['NIR', 'RED']).rename('NDVI')
    return ee.Image.cat([img, NDVI])


In [50]:
landsat = landsat.map(add_NDVI)

# Get data of pixels in area using sample function

In [49]:
def times_series_pixels_of_area(geometry,
                                collection, 
                                date_start=None,
                                duration_month=10, 
                                band='NDVI'):
    
    # a function for sampling over a geometry 
    def f(img):
        # the time of the image
        t = img.date().millis() 
        # the sample function return the data here NDVI and if the geometries = True, 
        # it also return the centers of the pixels it samples from. 
        # It samples from all the pixels on the image
        fc = img.select(band).sample(geometry,geometries=True) 
        # aggregate_array function get all the data here NDVI in the feature collection we built above 
        ndvi = fc.aggregate_array('NDVI')
        # these points are the centers of the pixels 
        points_cords = fc.geometry().coordinates()
        # now we add the below dictionary to the feature collection  consisting time, ndvi and coordinates and return it
        return fc.set({'millis': t,'NDVI':ndvi,'coordinates':points_cords})
    
    # This function on the other hand gets the property list in its argument and construct a dictionary for returning 
    def fc_to_dict(fc,prop_names=ee.List(['millis','system:index','NDVI','coordinates'])):
        prop_lists = fc.reduceColumns(reducer=ee.Reducer.toList().repeat(prop_names.size()),selectors=prop_names).get('list') 
        return ee.Dictionary.fromLists(prop_names, prop_lists)
   

    
    # here we want to select the band of the collection we give to this function for sampling and constrain the date range
    if date_start == None:
        col = collection.select(band).filterBounds(geometry)
    elif date_start != None:
        col = collection.select(band).filterBounds(geometry).filterDate(ee.Date(date_start),ee.Date(date_start).advance(duration_month,'month'))
    
    
    # map the sampling function on the collection 
    col = col.map(f)
    # make the dictionary and get it from the server
    res = fc_to_dict(col).getInfo()
    
    # res is the dictionary returned. it has 4 keys as belows 
    coords = res['coordinates']
    ndvis = res['NDVI']
    millis = res['millis']
    ids = res['system:index']

    # we do the following beacuse of a problem. The problem is when there is just one pixel in a specific date, 
    # the associated data in coords[j] is not like [x,y] but it is x,y i.e. it has length 2. for two pixels or greater on the 
    # other hand we have [[x1,y1],[x2,y2], ...]. Here we trasnform the one-pixels data appropriately. 
    for i,c in enumerate(coords):
        if len(c) == 2 and type(c[0]) == float:
            coords[i] = [c]
    

    # we construct the dataframe of all data 
    res_dicts = [{'coordinates':coords[i],'NDVI':ndvis[i],'time':millis[i],'id':ids[i]} for i in range(len(coords))]
    df = pd.concat([pd.DataFrame(d) for d in res_dicts], ignore_index=True) 
    
    
    # add longitude and lattitude separately  
    df['lon'] = df['coordinates'].apply(lambda x: x[0])
    df['lat'] = df['coordinates'].apply(lambda x: x[1])
    df = df.drop(columns=['coordinates'])
    
    # add relative times to df
    def add_date_info(df):
        df['Timestamp'] = pd.to_datetime(df['time'], unit='ms')
        df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
        df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
        df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
        df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
        return df
    add_date_info(df)
    
    return df

# Get the data from a shapefile
here we first construct a geopandas dataframe from a shapefile, then for any specific pasture for example we can use the above function to get the data 

In [ ]:
# for reading the shapefiles 
import shapefile
# for geo dataframes 
import geopandas

In [35]:
gdf = geopandas.read_file('./Cottonwood_Pastures/CTWD_Pastures.shp').to_crs(epsg="4326")

In [36]:
# a function to transform a set of coordinates to a Polygon object in GEE. 
def id_to_ee(gdf, pasture_id):
    xx, yy = gdf['geometry'].iloc[pasture_id].exterior.coords.xy
    x = xx.tolist()
    y = yy.tolist()
    coordinates = zip(x,y)
    return ee.Geometry.Polygon([i for i in coordinates])

In [38]:
gdf

,OBJECTID,Id,area,Shape_Leng,Shape_Area,Pasture,Pasture_WP,geometry
0,0,0,0.0,0.0,284.543166,10,10,"POLYGON ((-101.87369 43.94697, -101.87364 43.9..."
1,0,0,0.0,0.0,230.760450,11,11,"POLYGON ((-101.87051 43.95497, -101.86785 43.9..."
2,0,0,0.0,0.0,51.885013,14,14,"POLYGON ((-101.86788 43.95139, -101.86030 43.9..."
3,0,0,0.0,0.0,88.258233,13,13,"POLYGON ((-101.86793 43.94833, -101.86026 43.9..."
4,0,0,0.0,0.0,80.723940,12,12,"POLYGON ((-101.86031 43.94276, -101.86036 43.9..."
5,0,0,0.0,0.0,134.932503,15,15,"POLYGON ((-101.86030 43.95139, -101.86788 43.9..."
6,0,0,0.0,0.0,335.638931,7,7,"POLYGON ((-101.84245 43.93707, -101.84242 43.9..."
7,0,0,0.0,0.0,134.022666,2,2b,"POLYGON ((-101.84057 43.95112, -101.84057 43.9..."
8,0,0,0.0,0.0,73.739023,1,1,"POLYGON ((-101.86030 43.95139, -101.85429 43.9..."
9,0,0,0.0,0.0,162.932180,3,3b,"POLYGON ((-101.85289 43.94776, -101.85168 43.9..."


In [58]:
gdf['centers'] = gdf['geometry'].to_crs(epsg="4326").centroid

/var/folders/sz/44hp86gj0lq16_fywprf9r580000gn/T/ipykernel_22926/1478759676.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centers'] = gdf['geometry'].to_crs(epsg="4326").centroid


In [61]:
# explore different pastures and see them on the map 
gdf.explore()

In [40]:
pasture_id0 = id_to_ee(gdf,0)

In [41]:
# see the area of pasture with id = 13
id_to_ee(gdf,13).area(1).getInfo()

118212.02356193292

In [51]:
# get all time landast NDVI for all the pixels on pasture 13
df13 = times_series_pixels_of_area(id_to_ee(gdf,13),landsat,date_start=None) 

In [52]:
df13

,NDVI,time,id,lon,lat,Timestamp,Year,Month,Day,DOY
0,0.076090,561919821159,1_1_1_1_2_LT04_032029_19871022,-101.859951,43.965609,1987-10-22 16:50:21.159,1987,10,22,295
1,0.075892,561919821159,1_1_1_1_2_LT04_032029_19871022,-101.859577,43.965618,1987-10-22 16:50:21.159,1987,10,22,295
2,0.076013,561919821159,1_1_1_1_2_LT04_032029_19871022,-101.859204,43.965628,1987-10-22 16:50:21.159,1987,10,22,295
3,0.055199,561919821159,1_1_1_1_2_LT04_032029_19871022,-101.858830,43.965637,1987-10-22 16:50:21.159,1987,10,22,295
4,0.075892,561919821159,1_1_1_1_2_LT04_032029_19871022,-101.859938,43.965339,1987-10-22 16:50:21.159,1987,10,22,295
...,...,...,...,...,...,...,...,...,...,...
203616,0.098164,1669829790977,2_LC09_033029_20221130,-101.856223,43.960847,2022-11-30 17:36:30.977,2022,11,30,334
203617,0.099088,1669829790977,2_LC09_033029_20221130,-101.855849,43.960837,2022-11-30 17:36:30.977,2022,11,30,334
203618,0.110061,1669829790977,2_LC09_033029_20221130,-101.855476,43.960826,2022-11-30 17:36:30.977,2022,11,30,334
203619,0.110771,1669829790977,2_LC09_033029_20221130,-101.855103,43.960816,2022-11-30 17:36:30.977,2022,11,30,334


# The precipitation data and tempreture data 

In [53]:
# we use PRISM data with spatial Resolution 4638.3 meters and daily time resolution 
pr_PRISM = ee.ImageCollection("OREGONSTATE/PRISM/AN81d") 

In [54]:
# get the precipitation and temperature data for a point
def get_df_precip_tmean(point,scale):    
    res = pr_PRISM.select(['tmean','ppt']).getRegion(point,scale)
    df = pd.DataFrame(res.getInfo())
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    def add_date_info(df):
        df['Timestamp'] = pd.to_datetime(df['time'], unit='ms')
        df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
        df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
        df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
        df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
        return df
    add_date_info(df)
    return df

In [ ]:
# get precipitation and temperature data and reduce over a region 
def get_df_precip_tmean_reduce_region(geometry,reducer = ee.Reducer.mean()):    
    # get the collection and bands 
    c = pr_PRISM.select(['tmean','ppt'])
    
    # create a reduce function 
    def create_reduce_func(geometry, reducer):
        def f(img):
            img = img.select(['tmean','ppt'])
            mean = img.reduceRegion(reducer = reducer, geometry = geometry)
            return ee.Feature(geometry,mean).set({'millis': img.date().millis()})
        return f
    
    # a function that helps to turn properties data to dictionary
    def fc_to_dict(fc):
        prop_names = fc.first().propertyNames()
        prop_lists = fc.reduceColumns(reducer=ee.Reducer.toList().repeat(prop_names.size()),
                                      selectors=prop_names).get('list') 
        return ee.Dictionary.fromLists(prop_names, prop_lists)
    
    f = create_reduce_func(geometry,reducer)
    col = c.map(f)
    d = fc_to_dict(col).getInfo()
    
    df = pd.DataFrame(d)
    def add_date_info(df):
        df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
        df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
        df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
        df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
        df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
        return df
    add_date_info(df)
    return df

In [84]:
# a function to transform a set of coordinates to a Polygon object in GEE. 
def id_to_ee_center_point(gdf, pasture_id):
    xx, yy = gdf['centers'].iloc[pasture_id].coords.xy
    x = xx.tolist()
    y = yy.tolist()
    return ee.Geometry.Point(*x,*y)

In [87]:
poi = id_to_ee_center_point(gdf,13);

In [88]:
df_precip_tmean_point  = get_df_precip_tmean(poi,30)

In [151]:
df_precip_tmean_point

,id,longitude,latitude,time,tmean,ppt,Timestamp,Year,Month,Day,DOY
0,19810101,-101.858039,43.963056,347112000000,1.805,0,1980-12-31 12:00:00,1980,12,31,366
1,19810102,-101.858039,43.963056,347198400000,-1.891,0,1981-01-01 12:00:00,1981,1,1,1
2,19810103,-101.858039,43.963056,347284800000,-0.562,0,1981-01-02 12:00:00,1981,1,2,2
3,19810104,-101.858039,43.963056,347371200000,-3.732,0,1981-01-03 12:00:00,1981,1,3,3
4,19810105,-101.858039,43.963056,347457600000,-4.75,0,1981-01-04 12:00:00,1981,1,4,4
...,...,...,...,...,...,...,...,...,...,...,...
15438,20230409,-101.858039,43.963056,1680955200000,7.3786,0,2023-04-08 12:00:00,2023,4,8,98
15439,20230410,-101.858039,43.963056,1681041600000,11.344999,0,2023-04-09 12:00:00,2023,4,9,99
15440,20230411,-101.858039,43.963056,1681128000000,13.9935,0,2023-04-10 12:00:00,2023,4,10,100
15441,20230412,-101.858039,43.963056,1681214400000,21.186199,0,2023-04-11 12:00:00,2023,4,11,101


# Geopandas and some visualization and understanding

In [90]:
df13

,NDVI,time,id,lon,lat,Timestamp,Year,Month,Day,DOY
0,0.076090,561919821159,1_1_1_1_2_LT04_032029_19871022,-101.859951,43.965609,1987-10-22 16:50:21.159,1987,10,22,295
1,0.075892,561919821159,1_1_1_1_2_LT04_032029_19871022,-101.859577,43.965618,1987-10-22 16:50:21.159,1987,10,22,295
2,0.076013,561919821159,1_1_1_1_2_LT04_032029_19871022,-101.859204,43.965628,1987-10-22 16:50:21.159,1987,10,22,295
3,0.055199,561919821159,1_1_1_1_2_LT04_032029_19871022,-101.858830,43.965637,1987-10-22 16:50:21.159,1987,10,22,295
4,0.075892,561919821159,1_1_1_1_2_LT04_032029_19871022,-101.859938,43.965339,1987-10-22 16:50:21.159,1987,10,22,295
...,...,...,...,...,...,...,...,...,...,...
203616,0.098164,1669829790977,2_LC09_033029_20221130,-101.856223,43.960847,2022-11-30 17:36:30.977,2022,11,30,334
203617,0.099088,1669829790977,2_LC09_033029_20221130,-101.855849,43.960837,2022-11-30 17:36:30.977,2022,11,30,334
203618,0.110061,1669829790977,2_LC09_033029_20221130,-101.855476,43.960826,2022-11-30 17:36:30.977,2022,11,30,334
203619,0.110771,1669829790977,2_LC09_033029_20221130,-101.855103,43.960816,2022-11-30 17:36:30.977,2022,11,30,334


In [114]:
# get some info about the data, we can see for example how many pixels we have in the geometry 
df13[(df13['DOY']==139)&(df13['Year']==2000)].nunique()

NDVI         246
time           2
id             2
lon          133
lat          133
Timestamp      2
Year           1
Month          1
Day            1
DOY            1
dtype: int64

In [115]:
# turn it into a geopandas dataframe
gdf13 = geopandas.GeoDataFrame(df13)

In [116]:
# add geometries to the dataframe
gdf13.set_geometry(
    geopandas.points_from_xy(gdf13['lon'], gdf13['lat'],crs=4326),
    inplace=True)

gdf13

,NDVI,time,id,lon,lat,Timestamp,Year,Month,Day,DOY,geometry
0,0.076090,561919821159,1_1_1_1_2_LT04_032029_19871022,-101.859951,43.965609,1987-10-22 16:50:21.159,1987,10,22,295,POINT (-101.85995 43.96561)
1,0.075892,561919821159,1_1_1_1_2_LT04_032029_19871022,-101.859577,43.965618,1987-10-22 16:50:21.159,1987,10,22,295,POINT (-101.85958 43.96562)
2,0.076013,561919821159,1_1_1_1_2_LT04_032029_19871022,-101.859204,43.965628,1987-10-22 16:50:21.159,1987,10,22,295,POINT (-101.85920 43.96563)
3,0.055199,561919821159,1_1_1_1_2_LT04_032029_19871022,-101.858830,43.965637,1987-10-22 16:50:21.159,1987,10,22,295,POINT (-101.85883 43.96564)
4,0.075892,561919821159,1_1_1_1_2_LT04_032029_19871022,-101.859938,43.965339,1987-10-22 16:50:21.159,1987,10,22,295,POINT (-101.85994 43.96534)
...,...,...,...,...,...,...,...,...,...,...,...
203616,0.098164,1669829790977,2_LC09_033029_20221130,-101.856223,43.960847,2022-11-30 17:36:30.977,2022,11,30,334,POINT (-101.85622 43.96085)
203617,0.099088,1669829790977,2_LC09_033029_20221130,-101.855849,43.960837,2022-11-30 17:36:30.977,2022,11,30,334,POINT (-101.85585 43.96084)
203618,0.110061,1669829790977,2_LC09_033029_20221130,-101.855476,43.960826,2022-11-30 17:36:30.977,2022,11,30,334,POINT (-101.85548 43.96083)
203619,0.110771,1669829790977,2_LC09_033029_20221130,-101.855103,43.960816,2022-11-30 17:36:30.977,2022,11,30,334,POINT (-101.85510 43.96082)


In [121]:
# visualize the unique centers of all pixels in the dataframe
points_df = geopandas.GeoDataFrame(gdf13.drop_duplicates(subset=['geometry']))
points_df['geometry'].explore()

In [136]:
points_df.nunique()

NDVI         111
time           2
id             2
lon          268
lat          268
Timestamp      2
Year           2
Month          2
Day            2
DOY            2
geometry     268
dtype: int64

In [129]:
gdf13_year_doy = geopandas.GeoDataFrame(df13[(df13['DOY']==139)&(df13['Year']==2000)])

In [134]:
gdf13_year_doy.nunique()

NDVI         246
time           2
id             2
lon          133
lat          133
Timestamp      2
Year           1
Month          1
Day            1
DOY            1
geometry     133
dtype: int64

In [137]:
# here we can see the pattern of centers of the pixels for a specific day of year is actually a grid
gdf13_year_doy['geometry'].explore()

In [138]:
# this function helps to mean over the region and return dataframe of the pasture over time
def mean_over_region(df):
    df_mean = df[['Year','Month','Day','DOY','NDVI']].groupby(['Year','Month','Day','DOY']).mean().reset_index()
    df_mean['Timestamp'] = pd.to_datetime(df_mean['Year'] * 1000 + df_mean['DOY'], format='%Y%j')
    return df_mean

In [140]:
df_mean_pasture = mean_over_region(gdf13)

In [149]:
# here we visualize the mean dataframe over the pasture
alt.Chart(df_mean_pasture).mark_bar(size=1).encode(
alt.X("Timestamp:T"), alt.Y("NDVI:Q"),
    tooltip=[
        alt.Tooltip('DOY:O', title='DOY'),
        alt.Tooltip('NDVI:Q', title='NDVI')]
).properties(width=2000, height=400)

alt.Chart(...)

#  merging precipitation, temperature and NDVI dataframes

In [ ]:
# simplify TImestamp for merging later with ndvi dataframe
df_precip_tmean_point['Timestamp'] = pd.to_datetime(df_precip_tmean_point['Year'] * 1000 + df_precip_tmean_point['DOY'], format='%Y%j')
df_precip_tmean_point

In [153]:
df_mean_pasture

,Year,Month,Day,DOY,NDVI,Timestamp
0,1982,11,16,320,0.079252,1982-11-16
1,1984,4,12,103,0.050866,1984-04-12
2,1984,5,5,126,0.014998,1984-05-05
3,1984,5,14,135,0.163162,1984-05-14
4,1984,6,6,158,0.198883,1984-06-06
...,...,...,...,...,...,...
1142,2023,2,11,42,0.083553,2023-02-11
1143,2023,2,19,50,0.087755,2023-02-19
1144,2023,2,27,58,0.136045,2023-02-27
1145,2023,3,14,73,0.037038,2023-03-14


In [155]:
df_merged = pd.merge_ordered(df_precip_tmean_point, df_mean_pasture, left_by=['Year','Month','Day','DOY'])

In [156]:
# note that NDVI is sparse relative to ppt and tmean which are daily
df_merged

,id,longitude,latitude,time,tmean,ppt,Timestamp,Year,Month,Day,DOY,NDVI
0,19810101,-101.858039,43.963056,347112000000,1.805,0,1980-12-31,1980,12,31,366,NaN
1,19810102,-101.858039,43.963056,347198400000,-1.891,0,1981-01-01,1981,1,1,1,NaN
2,19810103,-101.858039,43.963056,347284800000,-0.562,0,1981-01-02,1981,1,2,2,NaN
3,19810104,-101.858039,43.963056,347371200000,-3.732,0,1981-01-03,1981,1,3,3,NaN
4,19810105,-101.858039,43.963056,347457600000,-4.75,0,1981-01-04,1981,1,4,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
15438,20230409,-101.858039,43.963056,1680955200000,7.3786,0,2023-04-08,2023,4,8,98,NaN
15439,20230410,-101.858039,43.963056,1681041600000,11.344999,0,2023-04-09,2023,4,9,99,NaN
15440,20230411,-101.858039,43.963056,1681128000000,13.9935,0,2023-04-10,2023,4,10,100,NaN
15441,20230412,-101.858039,43.963056,1681214400000,21.186199,0,2023-04-11,2023,4,11,101,NaN


In [157]:
# computing GDD and adding it to the dataframe
t_base = 5 #Celsius unit
# first compute GD
df_merged['GD'] = df_merged['tmean'] - t_base
# then reset to zero the ones with negative values 
df_merged['GD'] = df_merged['GD'].apply(lambda x: np.max([x,0]))
# then adding cummulative GDD which starts at the begining of a year to the end of the year
df_merged['GDD'] = df_merged.groupby(df_merged['Timestamp'].dt.year)['GD'].cumsum()
df_merged

,id,longitude,latitude,time,tmean,ppt,Timestamp,Year,Month,Day,DOY,NDVI,GD,GDD
0,19810101,-101.858039,43.963056,347112000000,1.805,0,1980-12-31,1980,12,31,366,NaN,0.000000,0.000000
1,19810102,-101.858039,43.963056,347198400000,-1.891,0,1981-01-01,1981,1,1,1,NaN,0.000000,0.000000
2,19810103,-101.858039,43.963056,347284800000,-0.562,0,1981-01-02,1981,1,2,2,NaN,0.000000,0.000000
3,19810104,-101.858039,43.963056,347371200000,-3.732,0,1981-01-03,1981,1,3,3,NaN,0.000000,0.000000
4,19810105,-101.858039,43.963056,347457600000,-4.75,0,1981-01-04,1981,1,4,4,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15438,20230409,-101.858039,43.963056,1680955200000,7.3786,0,2023-04-08,2023,4,8,98,NaN,2.378600,8.105199
15439,20230410,-101.858039,43.963056,1681041600000,11.344999,0,2023-04-09,2023,4,9,99,NaN,6.344999,14.450199
15440,20230411,-101.858039,43.963056,1681128000000,13.9935,0,2023-04-10,2023,4,10,100,NaN,8.993500,23.443698
15441,20230412,-101.858039,43.963056,1681214400000,21.186199,0,2023-04-11,2023,4,11,101,NaN,16.186199,39.629898


In [162]:
# scale for better visualization 
df_merged['NDVI_scaled'] = df_merged['NDVI'] * 150
df_merged['GDD_scaled'] = df_merged['GDD']/100

In [163]:
tmean = alt.Chart(df_merged.iloc[5000:10000]).mark_bar(color='red',size=.2).encode(
alt.X("Timestamp:T"), alt.Y("tmean:Q"),
    tooltip=[
        alt.Tooltip('DOY:O', title='DOY'),
        alt.Tooltip('tmean:Q', title='tmean')]
)

precip = alt.Chart(df_merged.iloc[5000:10000]).mark_bar(color='blue',size=1).encode(
    alt.X("Timestamp:T"), alt.Y("ppt:Q"),
    tooltip=[
        alt.Tooltip('DOY:O', title='DOY'),
        alt.Tooltip('ppt:Q', title='ppt')]
)

ndvi = alt.Chart(df_merged.iloc[5000:10000]).mark_point(color='black',size=5).encode(
    alt.X("Timestamp:T"), alt.Y("NDVI_scaled:Q"),
    tooltip=[
        alt.Tooltip('DOY:O', title='DOY'),
        alt.Tooltip('NDVI:Q', title='NDVI')]
)

GDD = alt.Chart(df_merged.iloc[5000:10000]).mark_point(size=1).encode(
alt.X("Timestamp:T"), alt.Y("GDD_scaled:Q"),
    tooltip=[
        alt.Tooltip('DOY:O', title='DOY'),
        alt.Tooltip('GDD:Q', title='GDD')]
)



(tmean + precip + ndvi + GDD).properties(width=2000, height=600)


alt.LayerChart(...)